In [2]:
# string
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

# llm.invoke("What is the capital of the France?")

In [9]:
# prompt template
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the capital only.",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})
print(prompt)

ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))
print(ai_message)

output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))

text='What is the capital of France? Return the name of the capital only.'
content='Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-19T06:35:06.000064Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2357376576, 'load_duration': 1836031962, 'prompt_eval_count': 40, 'prompt_eval_duration': 388000000, 'eval_count': 3, 'eval_duration': 131000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-755d962f-88ec-466c-b59d-4cf7a13f1c13-0' usage_metadata={'input_tokens': 40, 'output_tokens': 3, 'total_tokens': 43}


In [10]:
# AIMessage는 BaseMessage 중 하나
ai_message

AIMessage(content='Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-19T06:35:06.000064Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2357376576, 'load_duration': 1836031962, 'prompt_eval_count': 40, 'prompt_eval_duration': 388000000, 'eval_count': 3, 'eval_duration': 131000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-755d962f-88ec-466c-b59d-4cf7a13f1c13-0', usage_metadata={'input_tokens': 40, 'output_tokens': 3, 'total_tokens': 43})

In [11]:
# answer는 string
answer

'Paris.'

In [21]:
# 개발할 때 Json도 많이 씀
# Json output parser도 있음
# 하지만 Json을 쓰는 것보다는 Pydantic을 쓰는 것이 좋음(꼭)
# 그래야 우리가 원하는 답변을 안정적으로 일정하게 받아낼 수 있음

from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

structured_llm = llm.with_structured_output(CountryDetail)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""
    Return the detail information of the {country}:
    - Capital
    - Population
    - Language
    - Currency
    return it in JSON format. and return the JSON dictionary only.
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({"country": "France"})
# output_parser = JsonOutputParser()
json_ai_message = structured_llm.invoke(country_detail_prompt.invoke({"country": "France"}))
# output_parser.invoke(json_ai_message)


In [24]:
(json_ai_message)

CountryDetail(capital='Paris', population=65275977, language='French', currency='Euro')

In [26]:
print(json_ai_message.capital)
print(json_ai_message.population)
print(json_ai_message.language)
print(json_ai_message.currency)




Paris
65275977
French
Euro


In [27]:
# json으로 만들기
json_ai_message.model_dump()

{'capital': 'Paris',
 'population': 65275977,
 'language': 'French',
 'currency': 'Euro'}